In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import scipy.io as sio

In [ ]:
from utils import f_rescale_dataset, f_train_test_split, f_get_mask, f_get_all_masks, f_Residuals
# from utils import f_reshape_training_data, f_rotate, f_translate, f_resize, f_augment_img, f_augment_dataset
from utils import f_reshape_training_data, f_rotate_and_zoom, f_random_crop, f_rotate_and_zoom_all, f_crop_all, f_flip_all, f_augment_dataset2

In [ ]:
N = 4
batch_size = N
ngpus = 1
num_workers = 1

In [ ]:
# import scipy.io as sio

# %cd /Users/Pracioppo/Desktop/VPTR/GA

# GA_data_FAF = sio.loadmat('GA_data_orig_256.mat')["GA_data"].astype(np.float32)
# GA_data_mask = sio.loadmat('GA_data_256.mat')["GA_data_256"].astype(np.float32)

# print(np.shape(GA_data_FAF))
# print(np.shape(GA_data_mask))

In [ ]:
import scipy.io as sio
%cd /Users/Pracioppo/Desktop/VPTR/GA_Zubens_data

data = sio.loadmat('GA_data_Zubin_256.mat')["out_img"].astype(np.float32)
# data[:,:,0,:,:]
GA_data_FAF  = data[:,:,0,:,:]
GA_data_mask = data[:,:,1,:,:]

print(np.shape(GA_data_FAF))
print(np.shape(GA_data_mask))

In [ ]:
# %cd Y:\Zubin-Data-Paper-ForPeter\data_GA_reg_aug4
# GA_data_FAF = sio.loadmat('GA_data_Zubin_256_full_FAFs.mat')
# print(np.shape(GA_data_FAF))

In [ ]:
data_shape = np.shape(GA_data_FAF)
tot_data_num = data_shape[0]
num_time_steps = data_shape[1]
img_sz = data_shape[2]

In [ ]:
train_num = 56
test_num = tot_data_num - train_num

GA_data_FAF = f_rescale_dataset(GA_data_FAF)
train_data_FAF, test_data_FAF, train_GA_data_mask, test_GA_data_mask = f_train_test_split(GA_data_FAF,GA_data_mask,train_num)
train_data_mask, test_data_mask = f_get_all_masks(train_GA_data_mask,test_GA_data_mask,train_num,test_num,num_time_steps)

In [ ]:
for i in np.arange(56):
    plt.imshow(train_data_mask[i,:,0,:,:].squeeze())
    plt.show()

In [ ]:
for i in np.arange(10):
    plt.imshow(test_data_mask[i,:,0,:,:].squeeze())
    plt.show()

In [ ]:
train_data_mask.size()

In [ ]:
# Get residual images and rescale everything:

train_Residuals = f_Residuals(train_data_mask)
test_Residuals = f_Residuals(test_data_mask)

train_Residuals = train_Residuals - torch.min(train_Residuals)
train_Residuals = train_Residuals/torch.max(train_Residuals)
torch.max(torch.flatten(train_Residuals,start_dim=1),axis=1)[0]

test_Residuals = test_Residuals - torch.min(test_Residuals)
test_Residuals = test_Residuals/torch.max(test_Residuals)
# test_Residuals = (test_Residuals - torch.min(test_Residuals,axis=0)[0/])/torch.max(test_Residuals,axis=0)[0]

train_data_FAF_rescale = (train_data_FAF - torch.min(train_data_FAF))/torch.max(train_data_FAF)
test_data_FAF_rescale = (test_data_FAF - torch.min(test_data_FAF))/torch.max(test_data_FAF)
# train_data_FAF_rescale = (train_data_FAF - torch.min(train_data_FAF))/torch.max(torch.flatten(train_data_FAF,start_dim=1),axis=1)[0].view(train_num,1,1,1,1)
# test_data_FAF_rescale = (test_data_FAF - torch.min(test_data_FAF))/torch.max(torch.flatten(test_data_FAF,start_dim=1),axis=1)[0].view(test_num,1,1,1,1)

In [ ]:
# Construct the full dataset:

Train_Data = torch.concatenate((train_data_FAF_rescale,train_data_mask,train_Residuals),4)
Test_Data = torch.concatenate((test_data_FAF_rescale,test_data_mask,test_Residuals),4)
All_Data = torch.concatenate((Train_Data, Test_Data))

print(Train_Data.size())
print(Test_Data.size())
print(All_Data.size())

In [ ]:
# Reshape dataset:

train_data = torch.zeros(train_num,num_time_steps-1,2,img_sz,img_sz*3)
test_data = torch.zeros(test_num,num_time_steps-1,2,img_sz,img_sz*3)
all_data = torch.zeros(train_num+test_num,num_time_steps-1,2,img_sz,img_sz*3)

for i in np.arange(train_num):
    train_data[i,:,:,:,:] = f_reshape_training_data(Train_Data[i,0,:,:,:])
    
for i in np.arange(test_num):
    test_data[i,:,:,:,:] = f_reshape_training_data(Test_Data[i,0,:,:,:])
    
for i in np.arange(train_num + test_num):
    all_data[i,:,:,:,:] = f_reshape_training_data(All_Data[i,0,:,:,:])

print(train_data.size())
print(test_data.size())
print(all_data.size())

# x = all_data[0,0,1,:,512:].detach().cpu().numpy()
# plt.imshow(x)

In [ ]:
## Plot the full dataset

data = all_data.detach().cpu().numpy()
dims = all_data.size()
num_img = dims[0]

for i_t in np.arange(num_img)*10:

    print("Sample:", i_t)

    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    for idx, ax in enumerate(axes.flat):
        ax.imshow(data[i_t,idx,0,:,:])
        ax.set_title(f"Frame {idx + 1}")
        ax.axis("off")

    plt.show()
    
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    for idx, ax in enumerate(axes.flat):
        ax.imshow(data[i_t,idx,1,:,:])
        ax.set_title(f"Frame {idx + 1}")
        ax.axis("off")

    plt.show()

In [ ]:
def f_remove_data(data,del_idxs):

    cur_idx = 0
    ref_data = []
    for idx in del_idxs:
        data_frag = data[cur_idx:idx]
        cur_idx = idx+1
        if ref_data == []:
            ref_data = data_frag
        else:
            ref_data = torch.concatenate((ref_data,data_frag))

    data_frag = all_data[cur_idx:]
    ref_data = torch.concatenate((ref_data,data_frag))
    return ref_data

In [ ]:
del_idxs = [55, 60, 62]
ref_data = f_remove_data(all_data,del_idxs)
print(ref_data.size())

In [ ]:
# ## Plot the full dataset

# dims = ref_data.size()
# num_img = dims[0]

# for i_t in np.arange(num_img):
    
#     print("Sample:", i_t)

#     fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    
#     for idx, ax in enumerate(axes.flat):
#         ax.imshow(ref_data[i_t,idx,0,:,:])
#         ax.set_title(f"Frame {idx + 1}")
#         ax.axis("off")
    
#     plt.show()
    
#     fig, axes = plt.subplots(1, 3, figsize=(14, 4))

#     for idx, ax in enumerate(axes.flat):
#         ax.imshow(ref_data[i_t,idx,1,:,:])
#         ax.set_title(f"Frame {idx + 1}")
#         ax.axis("off")
    
#     plt.show()

In [ ]:
idxs = np.arange(len(ref_data))
np.random.shuffle(idxs)
shuff_data = ref_data[idxs]

train_data = shuff_data[0:-9]
test_data = shuff_data[-9:]

print(shuff_data.size())
print(train_data.size())
print(test_data.size())

In [ ]:
# ## Plot the train dataset

# dims = train_data.size()
# num_img = dims[0]

# for i_t in np.arange(num_img):
    
#     print("Sample:", i_t)

#     fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    
#     for idx, ax in enumerate(axes.flat):
#         ax.imshow(train_data[i_t,idx,0,:,:])
#         ax.set_title(f"Frame {idx + 1}")
#         ax.axis("off")
    
#     plt.show()
    
#     fig, axes = plt.subplots(1, 3, figsize=(14, 4))

#     for idx, ax in enumerate(axes.flat):
#         ax.imshow(train_data[i_t,idx,1,:,:])
#         ax.set_title(f"Frame {idx + 1}")
#         ax.axis("off")
    
#     plt.show()

In [ ]:
# ## Plot the test dataset

# dims = test_data.size()
# num_img = dims[0]

# for i_t in np.arange(num_img):
    
#     print("Sample:", i_t)

#     fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    
#     for idx, ax in enumerate(axes.flat):
#         ax.imshow(test_data[i_t,idx,0,:,:])
#         ax.set_title(f"Frame {idx + 1}")
#         ax.axis("off")
    
#     plt.show()
    
#     fig, axes = plt.subplots(1, 3, figsize=(14, 4))

#     for idx, ax in enumerate(axes.flat):
#         ax.imshow(test_data[i_t,idx,1,:,:])
#         ax.set_title(f"Frame {idx + 1}")
#         ax.axis("off")
    
#     plt.show()

In [ ]:
# Augment the dataset:

num_pass = 10
prob_flip = 0.5
prob_rot = 0.25
prob_crop = 0.25

train_data_augment = f_augment_dataset2(train_data, num_pass = num_pass, prob_flip = prob_flip, prob_rot = prob_rot, prob_crop = prob_crop)

np.shape(train_data_augment)

In [ ]:
test_data_augment = f_augment_dataset2(test_data, num_pass = num_pass, prob_flip = prob_flip, prob_rot = prob_rot, prob_crop = prob_crop)

np.shape(test_data_augment)

In [ ]:
# Should be around 1:
print(torch.max(torch.flatten(train_data_augment[:,:,:,:,0:256],start_dim=1),axis=1)[0])
print(torch.max(torch.flatten(train_data_augment[:,:,:,:,256:512],start_dim=1),axis=1)[0])
print(torch.max(torch.flatten(train_data_augment[:,:,:,:,512:768],start_dim=1),axis=1)[0])

In [ ]:
# Plot each of the sequential images for one random data example:

data_choice = np.random.choice(range(len(train_data_augment)), size=1)[0]
# data_choice = 1

# Construct a figure on which we will visualize the images.
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for idx, ax in enumerate(axes.flat):
    ax.imshow(train_data_augment[data_choice,idx,0,:,:])
    ax.set_title(f"Frame {idx + 1}")
    ax.axis("off")

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for idx, ax in enumerate(axes.flat):
    ax.imshow(train_data_augment[data_choice,idx,1,:,:])
    ax.set_title(f"Frame {idx + 1}")
    ax.axis("off")

In [ ]:
# ## Plot the full augmented dataset

# # Construct a figure on which we will visualize the images.

# dims = train_data.size()
# num_img = dims[0]

# # for i_t in np.arange(num_img*(num_pass-1))+num_img:
# for i_t in np.arange(len(train_data_augment)):
    
#     print("Iteration:", i_t)

#     fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    
#     for idx, ax in enumerate(axes.flat):
#         ax.imshow(train_data_augment[i_t,idx,0,:,:])
#         ax.set_title(f"Frame {idx + 1}")
#         ax.axis("off")
    
#     plt.show()
    
#     fig, axes = plt.subplots(1, 3, figsize=(14, 4))

#     for idx, ax in enumerate(axes.flat):
#         ax.imshow(train_data_augment[i_t,idx,1,:,:])
#         ax.set_title(f"Frame {idx + 1}")
#         ax.axis("off")
    
#     plt.show()

In [ ]:
# for i in np.arange(np.shape(train_data_augment)[0]):
#     plt.imshow(train_data_augment[i,2,1,:,:])
#     plt.show()

In [ ]:
sio.savemat('train_data_augment.mat',{"train_data": train_data_augment.numpy()})
# sio.savemat('test_data.mat',{"test_data": test_data.numpy()})
sio.savemat('test_data_augment.mat',{"test_data": test_data_augment.numpy()})

In [ ]:
# train_data_augment = torch.from_numpy(sio.loadmat('train_data_augment.mat')["train_data"].astype(np.float32))
# test_data  = torch.from_numpy(sio.loadmat('test_data.mat')["test_data"].astype(np.float32))

In [ ]:
print("Done")